In [ ]:
# default_exp algorithm

# Algorithm
> Method by Google X

In [ ]:
# exporti
from typing import NamedTuple

import jax
import jax.numpy as jnp
import matplotlib.pyplot as plt
from fastcore.basics import patch_to
from inverse_design.utils import batch_conv2d, conv2d, dilute
from matplotlib.colors import ListedColormap

In [ ]:
# export
UNASSIGNED = 0
VOID = 1
SOLID = 2
PIXEL_IMPOSSIBLE = 3
PIXEL_EXISTING = 4
PIXEL_POSSIBLE = 5
PIXEL_REQUIRED = 6
TOUCH_REQUIRED = 7
TOUCH_INVALID = 8
TOUCH_EXISTING = 9
TOUCH_VALID = 10
TOUCH_FREE = 11
TOUCH_RESOLVING = 12

In [ ]:
# export
class Design(NamedTuple):
    design: jnp.ndarray
    void_pixels: jnp.ndarray
    solid_pixels: jnp.ndarray
    void_touches: jnp.ndarray
    solid_touches: jnp.ndarray
    
    @property
    def shape(self):
        return self.design.shape
    
    def copy(self, **kwargs):
        kwargs = {name: kwargs.get(name, getattr(self, name)) for name in self._fields}
        return Design(*kwargs.values())

In [ ]:
# export
def new_design(shape):
    return Design(
        design=jnp.zeros(shape, dtype=jnp.uint8).at[:,:].set(UNASSIGNED),
        void_pixels=jnp.zeros(shape, dtype=jnp.uint8).at[:,:].set(PIXEL_POSSIBLE),
        solid_pixels=jnp.zeros(shape, dtype=jnp.uint8).at[:,:].set(PIXEL_POSSIBLE),
        void_touches=jnp.zeros(shape, dtype=jnp.uint8).at[:,:].set(TOUCH_VALID),
        solid_touches=jnp.zeros(shape, dtype=jnp.uint8).at[:,:].set(TOUCH_VALID),
    )

In [ ]:
# export
def circular_brush(diameter):
    radius = diameter / 2
    X, Y = jnp.mgrid[-radius:radius:1j*diameter,-radius:radius:1j*diameter]
    _int = lambda x: jnp.array(x, dtype=int)
    brush = _int(X)**2 + _int(Y)**2 < radius**2
    return brush

In [ ]:
# export
def notched_square_brush(width, notch):
    Z = jnp.ones((width, width), dtype=bool)
    notch = abs(notch)
    if notch > 0:
        Z = Z.at[:notch,:notch].set(False)
        Z = Z.at[:notch,-notch:].set(False)
        Z = Z.at[-notch:,:notch].set(False)
        Z = Z.at[-notch:,-notch:].set(False)
    return Z

In [ ]:
# export
def show_mask(brush):
    nx, ny = brush.shape
    _cmap = ListedColormap(colors={0: "#ffffff", 1: "#929292"}.values())
    ax = plt.gca()
    ax.set_yticks(jnp.arange(nx)+0.5, ["" for i in range(nx)])
    ax.set_xticks(jnp.arange(ny)+0.5, ["" for i in range(ny)])
    ax.set_yticks(jnp.arange(nx), [f"{i}" for i in range(nx)], minor=True)
    ax.set_xticks(jnp.arange(ny), [f"{i}" for i in range(ny)], minor=True)
    plt.grid(True, color='k')
    plt.imshow(brush, cmap=_cmap)

In [ ]:
show_mask(circular_brush(13));

In [ ]:
show_mask(notched_square_brush(5, 1))

In [ ]:
# export
def visualize(design):
    _cmap = ListedColormap(colors={UNASSIGNED: "#929292", VOID: "#cbcbcb", SOLID: "#515151", PIXEL_IMPOSSIBLE: "#8dd3c7", PIXEL_EXISTING: "#ffffb3", PIXEL_POSSIBLE: "#bebada", PIXEL_REQUIRED: "#fb7f72", TOUCH_REQUIRED: "#00ff00", TOUCH_INVALID: "#7fb1d3", TOUCH_EXISTING: "#fdb462", TOUCH_VALID: "#b3de69", TOUCH_FREE: "#fccde5", TOUCH_RESOLVING: "#e0e0e0"}.values(), name="cmap")
    nx, ny = design.design.shape
    fig, axs = plt.subplots(1, 5, figsize=(15,3*nx/ny))
    for i, title in enumerate(design._fields):
        ax = axs[i]
        ax.set_title(title.replace("_", " "))
        ax.imshow(design[i], cmap=_cmap, vmin=UNASSIGNED, vmax=TOUCH_RESOLVING)
        ax.set_yticks(jnp.arange(nx)+0.5, ["" for i in range(nx)])
        ax.set_xticks(jnp.arange(ny)+0.5, ["" for i in range(ny)])
        ax.set_yticks(jnp.arange(nx), [f"{i}" for i in range(nx)], minor=True)
        ax.set_xticks(jnp.arange(ny), [f"{i}" for i in range(ny)], minor=True)
        ax.set_xlim(-0.5, ny-0.5)
        ax.set_ylim(nx-0.5, -0.5)
        ax.grid(visible=True, which="major", c="k")

@patch_to(Design)
def _repr_html_(self):
    visualize(self)
    return ""

In [ ]:
my_brush = notched_square_brush(5, 1)
show_mask(my_brush)

In [ ]:
step1 = lambda: new_design((6, 8))
step1()

In [ ]:
# export

@jax.jit
def _find_free_touches(touches_mask, pixels_mask, brush):
    r = jnp.zeros_like(touches_mask, dtype=bool)
    m, n = r.shape
    i, j = jnp.arange(m), jnp.arange(n)
    I, J = [idxs.ravel() for idxs in jnp.meshgrid(i, j)]
    K = jnp.arange(m * n)
    R = jnp.broadcast_to(r[None, :, :], (m * n, m, n)).at[K, I, J].set(True)
    Rb = batch_conv2d(R, brush[None]) | pixels_mask
    free_idxs = (Rb == pixels_mask).all((1, 2))
    free_touches_mask = jnp.where(free_idxs[:, None, None], R, 0).sum(0, dtype=bool)
    return free_touches_mask ^ touches_mask


@jax.jit
def _find_required_pixels(pixel_map, brush):
    mask = (~pixel_map) & (~dilute(pixel_map, brush))
    return ~(dilute(mask, brush) | pixel_map)


@jax.jit
def add_void_touch(design, brush, pos):
    if isinstance(pos, tuple):
        void_touches_mask = design.void_touches.at[pos[0], pos[1]].set(TOUCH_EXISTING) == TOUCH_EXISTING
    else:
        assert pos.dtype == bool
        void_touches_mask = pos | (design.void_touches == TOUCH_EXISTING)
    void_pixel_mask = dilute(void_touches_mask, brush) | (design.design == VOID)
    required_void_pixel_mask = _find_required_pixels(void_pixel_mask, brush)
    diluted_mask = dilute(void_pixel_mask, brush)
    design_ = jnp.where(void_pixel_mask, VOID, design.design)
    free_void_touches_mask = _find_free_touches(void_touches_mask, void_pixel_mask | required_void_pixel_mask, brush)
    void_touches = jnp.where(design.void_touches == TOUCH_RESOLVING, TOUCH_VALID, design.void_touches)
    void_touches = jnp.where(void_touches_mask, TOUCH_EXISTING, void_touches)
    void_touches = jnp.where(free_void_touches_mask, TOUCH_FREE, void_touches)
    resolving_pixels = jnp.where(void_touches == TOUCH_VALID, dilute(required_void_pixel_mask, brush), False)
    void_touches = jnp.where(resolving_pixels, TOUCH_RESOLVING, void_touches)
    void_pixels = jnp.where(void_pixel_mask, PIXEL_EXISTING, design.void_pixels)
    void_pixels = jnp.where(required_void_pixel_mask, PIXEL_REQUIRED, void_pixels)
    solid_pixels =  jnp.where(void_pixel_mask, PIXEL_IMPOSSIBLE, design.solid_pixels)
    solid_pixels = jnp.where(required_void_pixel_mask, PIXEL_IMPOSSIBLE, solid_pixels)
    solid_touches = jnp.where(diluted_mask, TOUCH_INVALID, design.solid_touches)
    return Design(design_, void_pixels, solid_pixels, void_touches, solid_touches)

In [ ]:
step2 = lambda: add_void_touch(step1(), my_brush, (0, 6))
step2()

In [ ]:
# export
@jax.jit
def take_free_void_touches(design, brush):
    # originally: 
    # design = design.copy(void_touches=jnp.where(design.void_touches == TOUCH_FREE, TOUCH_EXISTING, design.void_touches))
    # ⬆ the above solution is not good. It does not resolve required touches if present, we need to actually use the brush:
    return add_void_touch(design, brush, (design.void_touches == TOUCH_FREE))

In [ ]:
step3 = lambda: take_free_void_touches(step2(), my_brush)
step3()

In [ ]:
# export

@jax.jit
def add_solid_touch(design, brush, pos):
    if isinstance(pos, tuple):
        solid_touches_mask = design.solid_touches.at[pos[0], pos[1]].set(TOUCH_EXISTING) == TOUCH_EXISTING
    else:
        assert pos.dtype == bool
        solid_touches_mask = pos | (design.solid_touches == TOUCH_EXISTING)
    solid_pixel_mask = dilute(solid_touches_mask, brush) | (design.design == SOLID)
    required_solid_pixel_mask = _find_required_pixels(solid_pixel_mask, brush)
    diluted_mask = dilute(solid_pixel_mask, brush)
    design_ = jnp.where(solid_pixel_mask, SOLID, design.design)
    free_solid_touches_mask = _find_free_touches(solid_touches_mask, solid_pixel_mask | required_solid_pixel_mask, brush)
    solid_touches = jnp.where(design.solid_touches == TOUCH_RESOLVING, TOUCH_VALID, design.solid_touches)
    solid_touches = jnp.where(solid_touches_mask, TOUCH_EXISTING, solid_touches)
    solid_touches = jnp.where(free_solid_touches_mask, TOUCH_FREE, solid_touches)
    resolving_pixels = jnp.where(solid_touches == TOUCH_VALID, dilute(required_solid_pixel_mask, brush), False)
    solid_touches = jnp.where(resolving_pixels, TOUCH_RESOLVING, solid_touches)
    solid_pixels = jnp.where(solid_pixel_mask, PIXEL_EXISTING, design.solid_pixels)
    solid_pixels = jnp.where(required_solid_pixel_mask, PIXEL_REQUIRED, solid_pixels)
    void_pixels =  jnp.where(solid_pixel_mask, PIXEL_IMPOSSIBLE, design.void_pixels)
    void_pixels = jnp.where(required_solid_pixel_mask, PIXEL_IMPOSSIBLE, void_pixels)
    void_touches = jnp.where(diluted_mask, TOUCH_INVALID, design.void_touches)
    return Design(design_, void_pixels, solid_pixels, void_touches, solid_touches)

In [ ]:
step4 = lambda: add_solid_touch(step3(), my_brush, (0, 0))
step4()

In [ ]:
step5 = lambda: add_void_touch(step4(), my_brush, (4, 6))
step5()

In [ ]:
step6 = lambda: take_free_void_touches(step5(), my_brush)
step6()

In [ ]:
step7 = lambda: add_void_touch(step6(), my_brush, (4, 4))
step7()

In [ ]:
step8 = lambda: take_free_void_touches(step7(), my_brush)
step8()

In [ ]:
step9 = lambda: add_void_touch(step8(), my_brush, (5, 0))
step9()

In [ ]:
step10 = lambda: take_free_void_touches(step9(), my_brush)
step10()

In [ ]:
step11 = lambda: add_void_touch(step10(), my_brush, (2, 5))
step11()

In [ ]:
step12 = lambda: take_free_void_touches(step11(), my_brush)
step12()